## 🟡 2. Chatbot com Gradio (Sem Memória)

**🔹 Enunciado:**  
Desenvolva uma interface web interativa com Gradio para o chatbot, permitindo que o usuário insira perguntas e receba respostas imediatamente, sem manter o histórico da conversa.

**🧩 Implementação:**  
A função `responder()` gera um prompt estático com exemplos fixos e insere a pergunta do usuário. A interface foi construída com `gr.Interface()` contendo um `Textbox` de entrada e saída.

**🚀 Resultados Esperados:**
- Interface visual simples e responsiva.
- Modelo responde de forma independente a cada pergunta, sem lembrar conversas anteriores.

[![Abrir no Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/flavioluizseixas/llm-first-steps/blob/main/Ex2.ipynb)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

In [ ]:
# Carregamento do modelo
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

In [ ]:
# Função de resposta
def responder(pergunta):
    prompt = f"""<|system|>Você é um assistente útil que responde sempre em português de forma clara e objetiva.<|end|>
<|user|>Qual é a capital da França?<|end|>
<|assistant|>A capital da França é Paris.<|end|>
<|user|>Quem descobriu o Brasil?<|end|>
<|assistant|>O Brasil foi descoberto por Pedro Álvares Cabral em 1500.<|end|>
<|user|>{pergunta}<|end|>
<|assistant|>"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    saida = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta = saida.split("<|assistant|>")[-1].strip()
    return resposta

In [ ]:
# Interface Gradio
iface = gr.Interface(
    fn=responder,
    inputs=gr.Textbox(label="Faça uma pergunta em português"),
    outputs=gr.Textbox(label="Resposta do Assistente"),
    title="Assistente TinyLlama em Português",
    description="Chatbot simples com modelo TinyLlama ajustado para responder em português."
)

iface.launch()
